In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
import os
import datetime as dt
from datetime import datetime
import bz2
import pickle
import _pickle as cPickle
import timeit
from __main__ import download_tickers, tickers

%matplotlib inline

ImportError: cannot import name 'download_tickers' from '__main__' (unknown location)

In [3]:
tickers = pd.read_csv('LSE.csv', header=None)

In [4]:
tickers

,0
0,02NG.L
1,0A0B.L
2,0A0C.L
3,0A1U.L
4,0A2K.L
...,...
6125,ZOE.L
6126,ZOL.L
6127,ZOO.L
6128,ZTF.L


In [5]:
tickers = tickers[0].to_list()

In [ ]:
#tickers = ['WEED.TO', 'NPINV.CO']
#timeit.timeit("stocks = download_tickers(tickers)", "from __main__ import download_tickers, tickers", number=1)
stocks = download_tickers(tickers[3000:4500])

[*********************100%***********************]  1500 of 1500 completed

158 Failed downloads:
- IPF2.L: No data found, symbol may be delisted
- FUSYQVCF.L: No data found, symbol may be delisted
- N30XWMN.L: No data found, symbol may be delisted
- FUSSCQV3.L: No data found, symbol may be delisted
- HDLVTW.L: No data found, symbol may be delisted
- HDLVEMN.L: No data found, symbol may be delisted
- MN1X.L: No data found, symbol may be delisted
- JMIN.L: No data found, symbol may be delisted
- GPVAN090.L: No data found, symbol may be delisted
- G12.L: No data found, symbol may be delisted
- ICOW.L: No data found, symbol may be delisted
- PIEXLN.L: No data found, symbol may be delisted
- G07.L: No data found, symbol may be delisted
- NLMK.L: No data found, symbol may be delisted
- G14.L: No data found, symbol may be delisted
- MNOD.L: No data found, symbol may be delisted
- FUSYQVCN.L: No data found, symbol may be delisted
- MED1.L: No data found, symbol may be delisted
- GPCSP003.L: N

In [18]:
import threading
threading.active_count()

5

In [6]:
stocks.loc[:, stocks.columns.get_level_values(1).isin(['Adj Close'])].iloc[:,3:11]

NameError: name 'stocks' is not defined

In [7]:
def download_tickers(tickers):
    # Fetch downloaded ticker from raw_data
    downloaded_tickers = [x[0:-5] if '.pbz2' in x else '' for x in os.listdir('raw_data/')]
    downloaded_tickers.remove('')
    # Check if tickers already are downloaded
    tickers_to_download = []
    for ticker in tickers:
        if ticker in downloaded_tickers:
            # If already downloaded, check if they need updates
            if update_ticker(ticker):
                tickers_to_download.append(ticker)
        else:
            tickers_to_download.append(ticker)

    download_dump(tickers_to_download)
    
    # Read tickers from json and return
    #return load_stocks(tickers)

In [8]:
def load_stocks(tickers):
    cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    index = pd.MultiIndex.from_product([tickers, cols], names=['Stock ticker', 'Data type'])
    if len(tickers) > 1:
        data = pd.concat([decompress_pickle('raw_data/' + x + '.pbz2') for x in tickers], axis=1, sort=True)
    else:
        data = decompress_pickle('raw_data/' + tickers[0] + '.pbz2')
    
    data.columns = index
    try:
        data.index = pd.to_datetime(data.index, unit='ms')
    except ValueError:
        data.index = pd.to_datetime(data.index)
    return data

In [9]:
def update_ticker(ticker):
    last_bday = last_weekday()
    newest_date = decompress_pickle('raw_data/' + ticker + '.pbz2').index[-1]
    if newest_date == float:
        newest_date = dt.datetime.fromtimestamp(newest_date/1000).date()
    if newest_date != last_bday:
        return True 
    else:
        return False

In [10]:
def download_dump(tickers):
    if tickers != []:
        data = yf.download(tickers, period='100y', group_by='tickers')
        if len(tickers) > 1:
            for ticker in tickers:
                    clean_data = clean_df(data[ticker])
                    #clean_data.to_json('raw_data/' + ticker + '.json', date_format='epoch', date_unit='ms')
                    compressed_pickle(ticker, data[ticker])
        else:
            clean_data = clean_df(data)
            #clean_data.to_json('raw_data/' + tickers[0] + '.json', date_format='epoch', date_unit='ms')
            compressed_pickle(tickers[0], data)

In [11]:
def last_weekday():
    todays_day = dt.date.today().day
    if todays_day in [5, 6]:
        return dt.date.today() - dt.timedelta(days=todays_day-4)
    else:
        return dt.date.today()


In [12]:
def clean_df(df):
    before = len(df)
    df = df.dropna(axis = 0, how='all')
    return df

In [13]:
def compressed_pickle(ticker, data):
    with bz2.BZ2File('raw_data/' + ticker + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)

In [14]:
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [15]:
test = decompress_pickle('raw_data/AAPL.pbz2')

In [16]:
test

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.101261,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095978,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088934,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.091135,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093777,73449600
...,...,...,...,...,...,...
2020-08-31,127.580002,131.000000,126.000000,129.039993,129.039993,225702700
2020-09-01,132.759995,134.800003,130.529999,134.179993,134.179993,152470100
2020-09-02,137.589996,137.979996,127.000000,131.399994,131.399994,200119000
